In [ ]:
# Imports
import astropy.constants as c
import astropy.units as u
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rc('font',size=20)

In [ ]:
# Parameters
outputDir='../../../output/cooling_test_new/'
filePrefix='SA_output_'
filePostfix='.h5'
snap='58'
firstFile=40
lastFile=40
xmin=1e12
xmax=1e15
ymin0=1e9
ymax0=1e14
ymin1=0
ymax1=0.2
baryonFrac=0.155 # Read from header!
Hubble_h=0.673
Central=True

In [ ]:
# First determine the size of the arrays that we need to hold the data
nGal=0
for iFile in range(firstFile,lastFile+1):
    # The following line closes the file at the end of the loop
    with h5py.File(outputDir+filePrefix+'%i'%iFile+filePostfix,'r') as f:
        nGal+=len(f[snap])
print('nGal=',nGal)

# Declare numpy arrays to hold the data
Type=np.empty(nGal)
Mvir=np.empty(nGal)
Rvir=np.empty(nGal)
Vvir=np.empty(nGal)
ColdGasRadius=np.empty(nGal)

# Now read in the data
iGal=0
for iFile in range(firstFile,lastFile+1):
    # The following line closes the file at the end of the loop
    with h5py.File(outputDir+filePrefix+'%i'%iFile+filePostfix,'r') as f:
        nGalFile=len(f[snap])
        Type[iGal:iGal+nGalFile]=f[snap]['Type']
        Mvir[iGal:iGal+nGalFile]=f[snap]['Mvir']
        Rvir[iGal:iGal+nGalFile]=f[snap]['Rvir']
        Vvir[iGal:iGal+nGalFile]=f[snap]['Vvir']
        ColdGasRadius[iGal:iGal+nGalFile]=f[snap]['ColdGasRadius']
        iGal+=nGalFile
        
if Central:
    Index=np.where(Type==0)
    Mvir=Mvir[Index]
    Rvir=Rvir[Index]
    Vvir=Vvir[Index]
    ColdGasRadius=ColdGasRadius[Index]

In [ ]:
tHalo=(Rvir*u.Mpc/Hubble_h/(Vvir*u.km/u.s)).to(u.yr)
tColdGas=(ColdGasRadius*u.Mpc/Hubble_h/(Vvir*u.km/u.s)).to(u.yr)

plt.figure(0,figsize=(8,6))
#plt.subplot(1,2,1)
#plt.loglog(Mvir*1e10,HotGas*1e10,'.')
#plt.xlabel(r'$M_\mathrm{200c}/h^{-1}M_\odot$')
#plt.ylabel(r'$M_\mathrm{HotGas}/h^{-1}M_\odot$')
#plt.plot([xmin,xmax],[baryonFrac*xmin,baryonFrac*xmax])
#plt.xlim([xmin,xmax])
#plt.ylim([ymin0,ymax0])
#plt.subplot(1,2,2)
plt.loglog(Mvir*1e10,tHalo,'.',label='Halo')
plt.loglog(Mvir*1e10,tColdGas,'.',label='ColdGas')
plt.xlabel(r'$M_\mathrm{200c}/h^{-1}M_\odot$')
plt.ylabel(r'$t_\mathrm{dyn}/\mathrm{yr}$')
#plt.plot([xmin,xmax],[baryonFrac,baryonFrac])
plt.xlim([xmin,xmax])
#plt.ylim([ymin1,ymax1])
plt.legend()

#plt.savefig('figs/HotGas_'+outputDir.split('/')[-2]+'.png')